<a href="https://colab.research.google.com/github/SedatDikbas/Python/blob/main/LTSM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikeras
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 4.5 MB/s eta 0:00:00


In [ ]:
#LSTM 100 Adımda Relu (Rectified Linear Unit) fonksiyonu ile çözüm. optimizer adam nöron sayısı 256 veri normalleştirme MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.layers import Activation
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from scikeras.wrappers import KerasRegressor



In [ ]:
# Veri setinizi yükleyin
df1 = pd.read_excel('veri_cengizhan.xlsx')
df = pd.DataFrame(df1)



In [ ]:
# 'DATE' sütununu tarih formatına dönüştürün
df['DATE'] = pd.to_datetime(df['DATE'])

# 'HOUR' sütununu saat formatına dönüştürün
df['HOUR'] = pd.to_datetime(df['HOUR'], format='%H:%M:%S').dt.time

# Tarih ve saat sütunlarını birleştirerek zaman damgası oluşturun
df['Timestamp'] = pd.to_datetime(df['DATE'].dt.strftime('%Y-%m-%d') + ' ' + df['HOUR'].astype(str))

# Zaman damgasını POSIX zaman damgasına dönüştürün
df['Timestamp'] = (df['Timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta(seconds=1)


In [ ]:
# HOLY sütununu bool veri tipine dönüştürün
df['HOLY'] = df['HOLY'].astype(bool)



In [ ]:
# NaN değerlerini sütunların medyanıyla doldurma
numeric_columns = df.select_dtypes(include=[np.number]).columns  # Sadece numerik sütunları seç
median_values = df[numeric_columns].median()  # Sadece numerik sütunların medyanını hesapla
df = df.fillna(value=median_values)  # NaN değerleri medyan değerleriyle doldur

# Bağımsız değişkenlerin seçimi
X = df[['Timestamp','HOLY', 'MCP-24', 'MCP-168', 'MCP-672','PISO','NGP', 'PIBO', 'SSOV', 'SBOV', 'MO', 'MB', 'TV', 'DER', 'TG', 'NG', 'HYD', 'LIG', 'RHYD', 'ICOAL', 'WIND', 'SOL', 'FUEL', 'GEO', 'ASPH', 'BCOAL', 'BIO', 'NAPH', 'LNG', 'IMEX', 'WAS', 'ANK', 'IST']]
y = df['MCP']  # Bağımlı değişken


In [ ]:
# Verileri normalleştirin
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# Eğitim ve test veri setlerini oluşturun
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# LSTM için veri şekillendirme
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

y_train = y_train.values.reshape((y_train.shape[0], 1))
y_test = y_test.values.reshape((y_test.shape[0], 1))

# X_train ve X_test'i DataFrame'e dönüştürme
X_train_df = pd.DataFrame(X_train.reshape(X_train.shape[0], X_train.shape[2]))
X_test_df = pd.DataFrame(X_test.reshape(X_test.shape[0], X_test.shape[2]))

y_train_df = pd.DataFrame(y_train.reshape(y_train.shape[0], y_train.shape[1]))
y_test_df = pd.DataFrame(y_test.reshape(y_test.shape[0], y_test.shape[1]))



In [ ]:
# LSTM modelini oluşturun
model = Sequential()
model.add(LSTM(256, input_shape=(1, X_train.shape[2])))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(1))
model.compile(loss='mean_absolute_error', optimizer='adam')


# Modeli eğitin
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Test veri seti üzerinde tahmin yapın
y_pred_lstm = model.predict(X_test)

# Tahminleri ters normalize edin
y_pred_lstm = stats.zscore(y_pred_lstm)
y_pred_lstm = y_pred_lstm * np.std(y_test) + np.mean(y_test)

#modelin performansını değerlendir
rmse_lstm = np.sqrt(mean_squared_error(y_test, y_pred_lstm))
r2_lstm = r2_score(y_test, y_pred_lstm)
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)

print("LSTM Model - Root Mean Squared Error:", rmse_lstm)
print("LSTM Model - R^2 Score:", r2_lstm)
print("LSTM Model - Mean Absolute Error:",mae_lstm)

Epoch 1/100
1754/1754 [==============================] - 21s 10ms/step - loss: 41.3316
Epoch 2/100
1754/1754 [==============================] - 19s 11ms/step - loss: 23.6834
Epoch 3/100
1754/1754 [==============================] - 18s 10ms/step - loss: 19.6105
Epoch 4/100
1754/1754 [==============================] - 19s 11ms/step - loss: 14.9226
Epoch 5/100
1754/1754 [==============================] - 18s 10ms/step - loss: 12.7213
Epoch 6/100
1754/1754 [==============================] - 20s 12ms/step - loss: 11.0550
Epoch 7/100
1754/1754 [==============================] - 18s 10ms/step - loss: 9.8880
Epoch 8/100
1754/1754 [==============================] - 18s 10ms/step - loss: 9.0780
Epoch 9/100
1754/1754 [==============================] - 19s 11ms/step - loss: 8.5170
Epoch 10/100
1754/1754 [==============================] - 17s 10ms/step - loss: 8.0878
Epoch 11/100
1754/1754 [==============================] - 18s 11ms/step - loss: 7.7269
Epoch 12/100
1754/1754 [======================

In [ ]:
# Excel dosyasına yazma
filepath = 'deneme.xlsx'
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
X_train_df.to_excel(writer, sheet_name='X_train')
X_test_df.to_excel(writer, sheet_name='X_test')

y_train_df.to_excel(writer, sheet_name='Y_train')
y_test_df.to_excel(writer, sheet_name='Y_test')


writer.save()


<ipython-input-19-2ae2e0df2edc>:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


#LSTM 100 Adımda Relu (Rectified Linear Unit) fonksiyonu ile çözüm. optimizer adam nöron sayısı 256 veri normalleştirme MinMaxScaler

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.layers import Activation


import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from scikeras.wrappers import KerasRegressor

# Veri setinizi yükleyin
df1 = pd.read_excel('data.xlsx')

df = pd.DataFrame(df1)

# 'DATE' sütununu tarih formatına dönüştürün
df['DATE'] = pd.to_datetime(df['DATE'])

# 'HOUR' sütununu saat formatına dönüştürün
df['HOUR'] = pd.to_datetime(df['HOUR'], format='%H:%M:%S').dt.time

# Tarih ve saat sütunlarını birleştirerek zaman damgası oluşturun
df['Timestamp'] = pd.to_datetime(df['DATE'].dt.strftime('%Y-%m-%d') + ' ' + df['HOUR'].astype(str))

# Zaman damgasını POSIX zaman damgasına dönüştürün
df['Timestamp'] = (df['Timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta(seconds=1)

# HOLY sütununu bool veri tipine dönüştürün
df['HOLY'] = df['HOLY'].astype(bool)

# NaN değerlerini sütunların medyanıyla doldurma
numeric_columns = df.select_dtypes(include=[np.number]).columns  # Sadece numerik sütunları seç
median_values = df[numeric_columns].median()  # Sadece numerik sütunların medyanını hesapla
df = df.fillna(value=median_values)  # NaN değerleri medyan değerleriyle doldur

# Bağımsız değişkenlerin seçimi
X = df[['Timestamp','HOLY', 'MCP-24', 'MCP-168', 'MCP-672','PISO','NGP', 'PIBO', 'SSOV', 'SBOV', 'MO', 'MB', 'TV', 'DER', 'TG', 'NG', 'HYD', 'LIG', 'RHYD', 'ICOAL', 'WIND', 'SOL', 'FUEL', 'GEO', 'ASPH', 'BCOAL', 'BIO', 'NAPH', 'LNG', 'IMEX', 'WAS', 'ANK', 'IST']]
y = df['MCP']  # Bağımlı değişken

# Verileri normalleştirin
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Eğitim ve test veri setlerini oluşturun
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# LSTM için veri şekillendirme
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

y_train = y_train.values.reshape((y_train.shape[0], 1))
y_test = y_test.values.reshape((y_test.shape[0], 1))

# X_train ve X_test'i DataFrame'e dönüştürme
X_train_df = pd.DataFrame(X_train.reshape(X_train.shape[0], X_train.shape[2]))
X_test_df = pd.DataFrame(X_test.reshape(X_test.shape[0], X_test.shape[2]))

y_train_df = pd.DataFrame(y_train.reshape(y_train.shape[0], y_train.shape[1]))
y_test_df = pd.DataFrame(y_test.reshape(y_test.shape[0], y_test.shape[1]))


# Excel dosyasına yazma
filepath = 'deneme.xlsx'
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
X_train_df.to_excel(writer, sheet_name='X_train')
X_test_df.to_excel(writer, sheet_name='X_test')

y_train_df.to_excel(writer, sheet_name='Y_train')
y_test_df.to_excel(writer, sheet_name='Y_test')


writer.save()


# LSTM modelini oluşturun
model = Sequential()
model.add(LSTM(256, input_shape=(1, X_train.shape[2])))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(1))
model.compile(loss='mean_absolute_error', optimizer='adam')


# Modeli eğitin
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Test veri seti üzerinde tahmin yapın
y_pred_lstm = model.predict(X_test)

# Tahminleri ters normalize edin
y_pred_lstm = stats.zscore(y_pred_lstm)
y_pred_lstm = y_pred_lstm * np.std(y_test) + np.mean(y_test)

#modelin performansını değerlendir
rmse_lstm = np.sqrt(mean_squared_error(y_test, y_pred_lstm))
r2_lstm = r2_score(y_test, y_pred_lstm)
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)

print("LSTM Model - Root Mean Squared Error:", rmse_lstm)
print("LSTM Model - R^2 Score:", r2_lstm)
print("LSTM Model - Mean Absolute Error:", mae_lstm)